In [1]:
import pandas as pd

In [2]:
# filepath of the CSV file
filepath = "../DATA/SILVER/TradeHistory/TradeHistory_data.csv"
# Read the CSV file into a DataFrame
df = pd.read_csv(filepath)
# Filter for specific segments
df = df[df["stock_name"].isin(["NIFTY-PE-24650-18JUL2024"])]
df

,datetime,exchange,segment,stock_name,scrip_code,side,amount,quantity,price,expiry_date
149,2024-07-15 13:23:50,FON,FO,NIFTY-PE-24650-18JUL2024,NIFTY,BUY,5762.50,50.0,115.25,2024-07-18
150,2024-07-15 14:18:31,FON,FO,NIFTY-PE-24650-18JUL2024,NIFTY,SELL,5787.50,50.0,115.75,2024-07-18
153,2024-07-16 13:57:37,FON,FO,NIFTY-PE-24650-18JUL2024,NIFTY,BUY,1941.25,25.0,77.65,2024-07-18
154,2024-07-16 13:57:37,FON,FO,NIFTY-PE-24650-18JUL2024,NIFTY,BUY,1942.50,25.0,77.70,2024-07-18
155,2024-07-18 09:47:40,FON,FO,NIFTY-PE-24650-18JUL2024,NIFTY,SELL,3512.50,50.0,70.25,2024-07-18
163,2024-07-18 14:25:45,FON,FO,NIFTY-PE-24650-18JUL2024,NIFTY,BUY,300.00,100.0,3.00,2024-07-18


In [3]:


# Calculate the total buy and sell quantities and prices
df = df.groupby(["segment", "stock_name", "side"]).agg(
    {"quantity": "sum", "amount": "sum", "price": "sum"}
)

# Reset the index to get stock names as a column
df.reset_index(inplace=True)

# Pivot the data for a better structure
df = df.pivot(
    index=["segment", "stock_name"],
    columns="side",
    values=["quantity", "amount", "price"],
).fillna(0)

df.columns = [
    "_".join(list(reversed(col))).strip().lower() for col in df.columns.values
]

df["buy_price"] = df["buy_amount"] / df["buy_quantity"]
df["sell_price"] = df["sell_amount"] / df["sell_quantity"]
df["pnl_amount"] = (df["sell_price"] - df["buy_price"]) * df["sell_quantity"]
df["holding_quantity"] = df["buy_quantity"] - df["sell_quantity"]
df["investment_amount"] = df["buy_price"] * df["holding_quantity"]
df["avg_price"] = df["investment_amount"] / df["holding_quantity"]
# Order by holding_quantity and stock_name
df = df.sort_values(by=["segment", "stock_name"])
# Reset the index to get stock names as a column
df = df.reset_index()
df = df.fillna(0)

# round the numbers
df = df.round(2)
df[
    [
        "segment",
        "stock_name",
        "buy_quantity",
        "buy_price",
        "buy_amount",
        "sell_quantity",
        "sell_price",
        "sell_amount",
        "pnl_amount",
        "holding_quantity",
        "investment_amount",
        "avg_price",
    ]
]

,segment,stock_name,buy_quantity,buy_price,buy_amount,sell_quantity,sell_price,sell_amount,pnl_amount,holding_quantity,investment_amount,avg_price
0,FO,NIFTY-PE-24650-18JUL2024,200.0,49.73,9946.25,100.0,93.0,9300.0,4326.88,100.0,4973.12,49.73


In [4]:
# filepath of the CSV file
filepath = "../DATA/GOLD/ProfitLoss/ProfitLoss_data.csv"

# Read the CSV file into a DataFrame
df_pnl = pd.read_csv(filepath)
df_pnl = df_pnl[df_pnl["stock_name"].isin(["NIFTY-PE-24650-18JUL2024"])]
df_pnl

,datetime,exchange,segment,stock_name,side,quantity,open_price,open_amount,close_price,close_amount,pnl_amount,pnl_percentage
81,2024-07-15 14:18:31,FON,FO,NIFTY-PE-24650-18JUL2024,LONG,50.0,115.25,5762.5,115.75,5787.5,25.00,0.43
82,2024-07-18 09:47:40,FON,FO,NIFTY-PE-24650-18JUL2024,LONG,50.0,77.68,3884.0,70.25,3512.5,-371.25,-9.56


In [5]:

# Calculate the total buy and sell quantities and prices
df_pnl = df_pnl.groupby(["segment", "stock_name"]).agg(
    {"pnl_amount": "sum"}
)

df_pnl = df_pnl.sort_values(by=["segment", "stock_name"])
# Reset the index to get stock names as a column
df_pnl.reset_index(inplace=True)
df_pnl

,segment,stock_name,pnl_amount
0,FO,NIFTY-PE-24650-18JUL2024,-346.25


In [6]:
# Rename the pnl_amount columns to distinguish them
df1 = df[["stock_name","pnl_amount"]].rename(columns={'pnl_amount': 'code_pnl'})
df2 = df_pnl[["stock_name","pnl_amount"]].rename(columns={'pnl_amount': 'my_pnl'})

# Perform an outer merge on the stock_name column
merged_df = pd.merge(df1, df2, on='stock_name', how='outer').fillna(0)
merged_df = merged_df[merged_df["code_pnl"] != merged_df["my_pnl"]]
merged_df

,stock_name,code_pnl,my_pnl
0,NIFTY-PE-24650-18JUL2024,4326.88,-346.25
